## Create sample grid for training data (code in neural.py)

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from Chempy.parameter import ModelParameters
from Chempy.cem_function import posterior_function_returning_predictions
from scipy.stats import norm as gaussian
import os
a = ModelParameters()

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
## This calculates a list of 5 trial values for each parameter around the prior value, as an array of 6 lists which will be combined
# Set the desired Gaussian sigma values in the widths parameter (values > prior sigma are used to fully explore parameter space)
# Parameter values are chosen that are evenly distributed in the Gaussian probability space (e.g. 16.7, 33, 50 etc. percentile points)

N = a.training_size # No. data points per parameter
widths = a.training_widths # Gaussian widths for parameters

# Create 1d grid of data points equally spaced in probability space 
prob = np.linspace(1/(N+1), 1-1/(N+1), N)
grids = [gaussian.ppf(prob) for _ in range(N+1)] # Normalize to unit Gaussian
norm_grid = np.array(np.meshgrid(*grids)).T.reshape(-1,N+1)

# Create grid in parameter space
param_grid = [item*widths+a.p0 for item in norm_grid]

# Save grids
directory = 'Neural/'
if not os.path.exists(directory):
    os.makedirs(directory)
np.save(directory+'training_norm_grid.npy',norm_grid)
np.save(directory+'training_param_grid.npy',param_grid)

In [ ]:
# Create abundance output
param_grid = param_grid[:10] # For testing
training_abundances = []
for i,item in enumerate(param_grid):
    abundances,_ = posterior_function_returning_predictions((item,a))
    training_abundances.append(abundances)
    if i%100 == 0:
        print("Calculating abundance set %d of %d" %(i,len(param_grid)))
              
# Save abundance table
np.save('Neural/training_abundances.npy', training_abundances)


NOTES

- All code is now in neural.py file
- Should put changeable parameters e.g. number of choices for each parameter in parameter.py file - DONE
-  Find nicer way of using all rows from grid - DONE
- Check whether to use Karakas 10 or Karakas 16 - Karakas 10 for testing
- Automate number of traceable elements - just copy that from code - DONE

*This may be a useful reference https://arxiv.org/abs/1502.01852, for recommendation of ReLU units, with w = np.random.randn(n) x np.sqrt(2/n) for initialized weights, from Stanford course*

*Another useful reference: https://arxiv.org/abs/1412.6980 for 'Adam' learning method viz Stanford course*


## Create Verification / Testing Datasets

In [4]:
a = ModelParameters()
names = ['verif','test']

In [15]:
for i,name in enumerate(names): # Create two identically distributed datasets
    length = a.verif_test_sizes[i]
    param_grid = []
    # Distribute data with prior widths
    for _ in range(length):
        param_grid.append(np.random.normal(size = len(a.p0), loc = a.p0,
                                           scale = a.test_widths))
    np.save("Neural/"+name+"_param_grid.npy",param_grid)
    
    model_abundances = []
    for j,jtem in enumerate(param_grid):
        abundances,_ = posterior_function_returning_predictions((jtem,a))
        model_abundances.append(abundances)
        if j%100 == 0:
            print("Calculating %s abundance set %d of %d" %(name,j,length))
            
    # Save abundance table
    np.save("Neural/"+name+"_abundances.npy",model_abundances)

Calculating verif abundance set 0 of 1000
gas reservoir is empty
gas reservoir is empty
gas reservoir is empty
gas reservoir is empty
gas reservoir is empty
gas reservoir is empty
gas reservoir is empty
Calculating verif abundance set 2 of 1000
Calculating verif abundance set 4 of 1000
Calculating verif abundance set 6 of 1000
Calculating verif abundance set 8 of 1000
Calculating test abundance set 0 of 1000
Calculating test abundance set 2 of 1000
Calculating test abundance set 4 of 1000
Calculating test abundance set 6 of 1000
gas reservoir is empty
gas reservoir is empty
gas reservoir is empty
gas reservoir is empty
gas reservoir is empty
gas reservoir is empty
gas reservoir is empty
gas reservoir is empty
gas reservoir is empty
gas reservoir is empty
gas reservoir is empty
gas reservoir is empty
Calculating test abundance set 8 of 1000


In [13]:
for i,name in enumerate(names): # Create two identically distributed datasets
    length = a.verif_test_sizes[i]
    norm_grid = []
    # Distribute data with prior width, but normalized with trial_widths as before
    for _ in range(length):
        norm_grid.append(np.random.normal(size = len(a.p0),
                                          scale = np.array(a.test_widths)/np.array(a.training_widths)))
     
    # Convert data into normalized form
    np.save("Neural/"+name+"_norm_grid.npy",norm_grid)
    
    # Find the actual abundance grid
    param_grid = [item*a.training_widths+a.p0 for item in norm_grid]
    np.save("Neural/"+name+"_param_grid.npy",param_grid)
    
    model_abundances = []
    for j,jtem in enumerate(param_grid[:10]):
        abundances,_ = posterior_function_returning_predictions((jtem,a))
        model_abundances.append(abundances)
        if j%2 == 0:
            print("Calculating %s abundance set %d of %d" %(name,j,length))
            
    # Save abundance table
    np.save("Neural/"+name+"_abundances.npy",model_abundances)

Calculating verif abundance set 0 of 1000
Calculating verif abundance set 2 of 1000
Calculating verif abundance set 4 of 1000
Calculating verif abundance set 6 of 1000
Calculating verif abundance set 8 of 1000


KeyboardInterrupt: 

## Create the Neural Network

In [ ]:
import numpy as np
import sys
import os
import torch # Import PyTorch
from torch.autograd import Variable
from Chempy.parameter import ModelParameters

In [ ]:
a = ModelParameters()

n_train = a.training_size**len(a.p0) # Training data points
n_neurons = a.neurons # No. neurons in layers

# Load pre-processed training data
tr_input = np.load('Neural/training_norm_grid.npy')
tr_output = np.load('Neural/training_abundances.npy')

# Calculate input dimension
dim_in = tr_input.shape[1]
dim_out = tr_output.shape[1]

# Convert to torch variables
tr_input = Variable(torch.from_numpy(tr_input)).type(torch.FloatTensor)
tr_output = Variable(torch.from_numpy(tr_output), requires_grad=False).type(torch.FloatTensor)

In [ ]:
## Calculate neural network - use one hidden layer and no. neurons specified in parameter.py.
# CHANGE these hyperparameters if needed
model = torch.nn.Sequential(
        torch.nn.Linear(dim_in, a.neurons),
        torch.nn.ReLU(),
        torch.nn.Linear(a.neurons,dim_out)
)
loss_fn = torch.nn.L1Loss(size_average=True)

# Use Adam optimizer with specified learning rate in parameter.py
optimizer = torch.optim.Adam(model.parameters(),lr = a.learning_rate)

In [ ]:
# Convergence counter
current_loss = 1000 # High inital loss set
count = 0
t = 0

In [ ]:
# Train the neural netowrk
for i in range(int(1e2)):
    pred_output = model(tr_input)
    loss = loss_fn(pred_output,tr_output)
    optimizer.zero_grad() # Zero gradients initially
    loss.backward() # Backpropagate
    optimizer.step() # Update via Adam method
    
    # Print cost
    if i % 10 == 0:
        print(i)
        print(loss.data[0])
    

In [ ]:
# Convert back to numpy
model_numpy = []
for param in model.parameters():
    model_numpy.append(param.data.numpy())
    
w_array_0 = model_numpy[0]
b_array_0 = model_numpy[1]
w_array_1 = model_numpy[2]
b_array_1 = model_numpy[3]

# Save parameters
np.savez("Numpy/neural_model.npz",
        w_array_0 = w_array_0,
        w_array_1 = w_array_1,
        b_array_0 = b_array_0,
        b_array_1 = b_array_1)

In [ ]:
def neural_output(test_input,neural_coeffs):
    """ This function will calculate the neural network predicted output.
    The neural network must be trained first.
    
    Inputs: 
    test_input - list containing unnormalized parameter values
    neural_coeffs - Neural network weights, stored in neural_model.npz file
    
    Output: Neural network abundance prediction
    """
    
    w_array_0,w_array_1,b_array_0,b_array_1 = neural_coeffs
    
    widths = 
    norm_data = [item/a.width]

**TO DO**
- Vary optimizer, ReLU function, L1Loss to see which gives best results
- Plot loss function against epoch
- Plot accuracy on verification dataset against epoch
- Vary learning_rate
- Plot accuracy on test data as 2D coloured plot in parameter space for each pair

**MUST change the predictions to output in NON normalized space**